In [2]:
import torch
import numpy as np
import pandas as pd 

In [4]:
test = torch.tensor([1,2,4,5,6,3,5,4,-1,-1,-1])
test

tensor([ 1,  2,  4,  5,  6,  3,  5,  4, -1, -1, -1])

In [14]:
test[test != -1]

tensor([1, 2, 4, 5, 6, 3, 5, 4])